In [1]:
import pandas as pd
import pyodbc
import numpy as np
import datetime as dt

In [2]:
# global vars
nights = [3,4,5,6,7,8,9,10,11,12,13,14]
load_matrix = [7,6,8,5,9,4,10,3,11,12,13,14]

# SQL servers credentials
live_samo = 'Driver={SQL Server};''Server=pck-sam-sql-04;''Database=TUI;''Trusted_Connection=yes;'
archive_samo = 'Driver={SQL Server};''Server=pvm-sam-sql-05.tui.local;''Database=TUI;''Trusted_Connection=yes;'
analytics = 'Driver={SQL Server};''Server=pvm-bi-sql-01;''Database=TUIOlap;''Trusted_Connection=yes;'

In [196]:
# function to pull data from sql table
def sql_pull(cred, sql_query):
    conn = pyodbc.connect(cred)
    cursor = conn.cursor()
    result = pd.read_sql_query(sql_query,conn)

    cursor.close()
    del cursor
    conn.close()
    
    return result



# pulling data from samo (currently fixed for KZN - DXB flights, for test purposes)
def flight_data():
    return sql_pull(live_samo, ' \
            SELECT frblock.Date AS \'Departure\', \
                   freight.lname AS \'Flight\', \
                   SUM(frblock.BCount + frblock.RCount) AS \'Seats\' \
            FROM frblock \
            LEFT JOIN freight \
                ON frblock.Freight = freight.inc \
            WHERE freight.lname IN (\'EK 131\',\'EK 132\',\'EK 133\',\'EK 134\') \
                  AND date >= CAST(GETDATE() +1 AS DATE) \
            GROUP BY frblock.Date, \
                     freight.lname ')



# function to pull data from sql and make pivot data frame
def flight_programm():
    # pulling data from sql
    data = flight_data()

    # vars , filters
    back = data[data.Flight.isin(['EK 131', 'EK 133'])]
    fp = data.copy()
    fp = fp[fp.Flight.isin(['EK 132', 'EK 134'])]

    # preparing df 
    for night in nights:
        fp[night] = 0

    # triple loop to place back flights in df correctly
    for departure in fp.Departure:
        for night in nights:
            for i in range(0, back.shape[0]):
                if departure + dt.timedelta(days = night) == back.iloc[i,0]:
                    number = back.iloc[i,2]
                    fp.loc[fp.Departure == departure, night] = number 
                    
    return fp



# back seats diagonal changer
def diagonal_changer(departure, night, block):

    local_night = night
    
    while night >= 3:
        
        
        if not forecast_el[forecast_el.Departure == departure].empty:
            forecast_el.loc[forecast_el.Departure == departure, [night]] = block
            
        night-= 1
        departure += dt.timedelta(days = 1)
        
    night = local_night +1
        
    while night <= 14:
    
        if not forecast_el[forecast_el.Departure == departure].empty:
            forecast_el.loc[forecast_el.Departure == departure, [night]] = block
            
        night+= 1
        departure -= dt.timedelta(days = 1)
        
        
        
def diagonal_changer_fc(departure, night, block):
    
    local_night = night

    while night >= 3:
        
        if not temp_df[temp_df.Departure == departure].empty:
            temp_df.loc[temp_df.Departure == departure, [night]] = block
            
        night-= 1
        departure += dt.timedelta(days = 1)

    night = local_night + 1    
        
    while night <= 14:
    
        if not temp_df[temp_df.Departure == departure].empty:
            temp_df.loc[temp_df.Departure == departure, [night]] = block
            
        night+= 1
        departure -= dt.timedelta(days = 1)
        
        
        
# main function to check and change  direct <-> back seats
def seats_calculator():
    for i in range(0, forecast_el.shape[0]):

        direct_seats = forecast_el.iloc[i, 2]
        direct_date = forecast_el.iloc[i, 0]

        if direct_seats == 0:
            pass
        
        else:
            for night in load_matrix:
                back_seats = forecast_el.iloc[i, night]

                if back_seats == 0:
                    pass

                elif back_seats >= direct_seats:
                    # updating local vars 
                    back_seats = back_seats - direct_seats
                    direct_seats = 0

                    # forecast_df - changing values
                    diagonal_changer(direct_date, night, back_seats)
                    forecast_el.iloc[i, 2] = 0

                else:
                    # updating local vars 
                    direct_seats = direct_seats - back_seats
                    back_seats = back_seats - back_seats

                    # forecast_df - changing values
                    diagonal_changer(direct_date, night, back_seats)
                    forecast_el.iloc[i, 2] = direct_seats
                    
                    

# forecast df with actual seats
def forecast():
    forecast_seats[nights] = 0
    
    for i in range(0, forecast_seats.shape[0]):

        direct_seats = forecast_seats.iloc[i, 2]
        direct_date = forecast_seats.iloc[i, 0]

        if direct_seats == 0:
            pass


        else:
            for night in load_matrix:
                back_seats = temp_df.iloc[i, night]

                if back_seats == 0:
                    pass


                elif back_seats >= direct_seats:
                    
                    # updating local vars 
                    back_seats = back_seats - direct_seats
                    step = direct_seats
                    direct_seats = 0

                    # forecast_df - changing values
                    forecast_seats.iloc[i, 2] = 0
                    forecast_seats.iloc[i,night] = step
                    diagonal_changer_fc(direct_date, night, back_seats)
                    
                else:
                    # updating local vars 
                    direct_seats = direct_seats - back_seats
                    step = back_seats
                    back_seats = back_seats - back_seats

                    # forecast_df - changing values
                    forecast_seats.iloc[i, 2] = direct_seats
                    forecast_seats.iloc[i,night] = step
                    diagonal_changer_fc(direct_date, night, back_seats)

In [197]:
# execution
fact_data = flight_programm()
forecast_el = fact_data.copy()
forecast_seats = fact_data.copy()
temp_df = fact_data.copy()

forecast()
seats_calculator()

forecast_seats['Seats'] = fact_data['Seats']
forecast_seats['Left'] = forecast_el['Seats']
forecast_seats['Sold'] = forecast_seats['Seats'] - forecast_seats['Left']

fact_data = fact_data.replace(0, '-')
forecast_el = forecast_el.replace(0, '-')
forecast_seats = forecast_seats.replace(0, '-')

forecast_seats = forecast_seats[['Departure', 'Flight', 'Seats', 'Sold', 'Left', 3,4,5,6,7,8,9,10,11,12,13,14]]

print(' -- Actual Data --')
display(fact_data)

print(' -- Forecast Back Seats --')
display(forecast_seats)

print(' -- Forecast EL --')
display(forecast_el)

 -- Actual Data --


,Departure,Flight,Seats,3,4,5,6,7,8,9,10,11,12,13,14
38,2021-03-27,EK 132,-,-,4,-,1,-,-,5,-,6,-,-,-
39,2021-03-28,EK 132,-,4,-,1,-,-,5,-,6,-,-,-,-
40,2021-03-29,EK 132,-,-,1,-,-,5,-,6,-,-,-,-,2
41,2021-03-30,EK 132,-,1,-,-,5,-,6,-,-,-,-,2,-
42,2021-04-01,EK 132,-,-,5,-,6,-,-,-,-,2,-,-,2
43,2021-04-03,EK 132,-,-,6,-,-,-,-,2,-,-,2,-,-
44,2021-04-04,EK 132,-,6,-,-,-,-,2,-,-,2,-,-,1
45,2021-04-05,EK 132,-,-,-,-,-,2,-,-,2,-,-,1,-
46,2021-04-06,EK 132,-,-,-,-,2,-,-,2,-,-,1,-,-
47,2021-04-08,EK 132,-,-,2,-,-,2,-,-,1,-,-,-,-


 -- Forecast Back Seats --


,Departure,Flight,Seats,Sold,Left,3,4,5,6,7,8,9,10,11,12,13,14
38,2021-03-27,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
39,2021-03-28,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
40,2021-03-29,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
41,2021-03-30,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
42,2021-04-01,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
43,2021-04-03,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
44,2021-04-04,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
45,2021-04-05,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
46,2021-04-06,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
47,2021-04-08,EK 132,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


 -- Forecast EL --


,Departure,Flight,Seats,3,4,5,6,7,8,9,10,11,12,13,14
38,2021-03-27,EK 132,-,-,4,-,-,-,4,5,-,6,-,-,-
39,2021-03-28,EK 132,-,4,-,-,-,4,5,-,6,-,-,-,-
40,2021-03-29,EK 132,-,-,-,-,4,5,-,6,-,-,-,-,4
41,2021-03-30,EK 132,-,-,-,4,5,-,6,-,-,-,-,4,-
42,2021-04-01,EK 132,-,4,5,-,6,-,-,-,-,4,-,5,2
43,2021-04-03,EK 132,-,-,6,-,-,-,-,2,-,5,2,-,-
44,2021-04-04,EK 132,-,6,-,-,-,-,2,-,-,2,-,-,6
45,2021-04-05,EK 132,-,-,-,-,-,2,-,-,2,-,-,6,-
46,2021-04-06,EK 132,-,-,-,-,2,-,-,2,-,-,1,-,1
47,2021-04-08,EK 132,-,-,2,-,-,2,-,-,1,-,1,-,-
